In [18]:
import ndjson

with open("input.ndjson") as f:
    data = ndjson.load(f)

track_id_by_key = {}
for row in data:
    if row["_type"] == "track":
        assert row["s3Key"] not in track_id_by_key
        track_id_by_key[row["s3Key"]] = row["_id"]


photo_id_by_key = {}
for row in data:
    if row["_type"] == "photo":
        assert row["s3Key"] not in photo_id_by_key
        photo_id_by_key[row["s3Key"]] = row["_id"]


def rename_location_as_trailhead(row):
    location = row.pop("location", None)
    if location is None:
        return
    row["trailhead"] = location


def convert_track_key_to_reference(row):
    track_key = row.pop("trackS3Key", None)
    if track_key is None:
        return
    track_id = track_id_by_key.get(track_key)
    if track_id is None:
        return
    row["track"] = {"_ref": track_id, "_type": "reference", "_weak": True}


def convert_cover_photo_key_to_reference(row):
    cover_photo_key = row.pop("coverPhotoS3Key", None)
    if cover_photo_key is None:
        return
    cover_photo_id = photo_id_by_key.get(cover_photo_key)
    if cover_photo_id is None:
        return
    row["coverPhoto"] = {
        "_ref": cover_photo_id,
        "_type": "reference",
        "_weak": True,
    }


def convert_report_photo_to_figure(row):
    report = row.get("report")
    if report is None:
        return
    for block in report:
        if block["_type"] != "photo":
            continue
        photo_key = block.pop("s3Key", None)
        if photo_key is None:
            continue
        photo_id = photo_id_by_key.get(photo_key)
        if photo_id is None:
            continue
        block.update(
            {
                "_type": "figure",
                "photo": {
                    "_ref": photo_id,
                    "_type": "reference",
                    "_weak": True,
                },
            }
        )


for row in data:
    if row["_type"] == "trip":
        rename_location_as_trailhead(row)
        convert_track_key_to_reference(row)
        convert_cover_photo_key_to_reference(row)
        convert_report_photo_to_figure(row)

with open("output.ndjson", "w") as f:
    ndjson.dump(data, f)